In [1]:
import pandas as pd
import numpy as np


from datetime import datetime

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from joblib import load, dump

import math

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [2]:
import import_ipynb


importing Jupyter notebook from load_normalize_dataset.ipynb
importing Jupyter notebook from reorder_dataset.ipynb
importing Jupyter notebook from create_split_balance_datasets.ipynb


In [ ]:
#@ Splitting the Dataset:
def split_Xy(data, column_y= []):    
    if (column_y == []):
        column_y=data.columns[-1]
    X = data.drop([column_y], axis=1,inplace=False)

    y = data[[column_y]].copy()

    return [X,y]

In [ ]:
def fillNormalizedData(X):
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value= 0)
    imp_mean.fit(X)

    Xfilled=pd.DataFrame(imp_mean.transform(X))
    Xfilled.columns=X.columns
    Xfilled.index=X.index
    return Xfilled

In [ ]:
def normalizeSomeData(data,column_y=[], categorical_columns = []):

   if (column_y == []):
    column_y=data.columns[-1]
    

   [data_mean,data_std] = load('01_import_data_normalization.joblib') 

   data_norm=data.drop([column_y], axis=1,inplace=False)

   data_norm=data.drop(categorical_columns, axis=1,inplace=False)    
   data_norm=(data_norm-data_mean[data_norm.columns])/data_std[data_norm.columns]


   data_norm[column_y] = data [column_y] 
   data_norm[categorical_columns] = data [categorical_columns] 
   data_norm = fillNormalizedData(data_norm)
   return data_norm

In [ ]:
def createDFfromCSV(data):
    data= data.set_index([data.columns[0],data.columns[1]])
    data.index = data.index.set_levels([ datetime.fromisoformat(data.index.levels[1][i]) for i in range(data.index.levels[1].shape[0])], level=1)
    
    return data

In [ ]:
def load_and_normalize_data():
    
    data = pd.read_csv(r'C:\Users\margh\OneDrive\Documenti\Adriaclim\Data\all_data.csv')

    data = createDFfromCSV(data)  
    data = data[['SSH', 'MSSH', 'any_damage']]
    
    train_set, test_set = train_test_split(data, test_size=0.2)
    train_set, validation_set = train_test_split(train_set, test_size=0.25)
    data_mean = train_set.mean()
    data_std = train_set.std()
    dump([data_mean,data_std] ,'01_import_data_normalization.joblib')     
    
    return [normalizeSomeData(data),normalizeSomeData(train_set),normalizeSomeData(validation_set),normalizeSomeData(test_set)]

In [ ]:
def reduce_training_set(train_set,percentage_rows=100 ):
    
    n_rows = train_set.shape[0]
    tmp=n_rows * percentage_rows/100
    new_row_number =math.trunc(tmp )
    train_set = train_set.iloc[0:new_row_number,:]

    return train_set    